In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, IntegerType

# SparkSession z Kafka
spark = SparkSession.builder \
    .appName("StreamingToParquet") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

# Schemat danych JSON
schema = StructType() \
    .add("id", IntegerType()) \
    .add("name", StringType()) \
    .add("message", StringType())

# Wczytanie danych ze strumienia Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka_streaming_lab:9092") \
    .option("subscribe", "spark-lab3-topic") \
    .option("startingOffsets", "latest") \
    .load()

# Parsowanie JSON
parsed = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Zapis do pliku Parquet
query = parsed.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("path", "/home/coder/project/parquet-output") \
    .option("checkpointLocation", "/home/coder/project/checkpoints/parquet") \
    .start()

query.awaitTermination()
